In [ ]:
import scipy
import os

import basic_mne_functions as bmf

In [ ]:
# epoch length
epoch = 5
# empty list to store sleep stages in 1d array where each 
# element is 1 epoch
sleep_states = []

# path to raw PSG files
raw_path = "C:/Users/andri/school/bio-informatics/internship/donders/data/human_test_data/input/raw_input"
# path to raw hypnogram annotation files
anno_path = "C:/Users/andri/school/bio-informatics/internship/donders/data/human_test_data/input/annotation_raw"
# path to the output for the .mat files
output_path = "C:/Users/andri/school/bio-informatics/internship/donders/data/human_test_data/mat_files/"

# iterate over both PSG and Hypno directories and get the file paths
for raw_file, anno_file in zip(os.listdir(raw_path), os.listdir(anno_path)):
        get_annotation = False
        subject = raw_file.split("/")[-1].split("-")[0]
        for channel in ["Fpz-Cz", "Pz-Oz", "horizontal", "submental"]:
            # extract raw data
            raw = bmf.extract_raw(f"{raw_path}/{raw_file}", channel)
            anno = bmf.extract_anno(f"{anno_path}/{anno_file}", raw)
            while get_annotation == False:
                sleep_events = bmf.get_sleep_states(raw, epoch)
                # create array from sleep events with only sleep stages
                for event in sleep_events:
                    sleep_states.append(int(event[-1]))
                scipy.io.savemat(
                    f"{output_path}{subject}_sleep_states.mat", 
                    {"States": sleep_states}
                )
                get_annotation = True
            # rename obscure channel names
            if channel == "horizontal":
                raw.rename_channels({"horizontal": "EOG"})
                channel = "EOG"
            elif channel == "submental":
                raw.rename_channels({"submental":"EMG"})
                channel = "EMG"
            # save numpy array to .mat file format
            scipy.io.savemat(
                f"{output_path}{subject}_{channel}.mat", 
                {channel: raw._data.reshape((-1,1))}
                )